# 2. QRBS uncertainty

In the *01_QRBS_for_dummies* jupyter notebook we provided the basic working of the **QRBS** library. We only focused on the categorical use of the library. The main functionality of the **QRBS** is using quantum circuits for modelling the evolution of indetermination propagating through an inferential network. In this case, the indetermination appears in two different ways:

* *Imprecision of the facts*: In this case, the facts are not categorical variables. Now the facts can be affected by a degree of belief, probability or even intensity. In the **QRBS** this will be modelled by assigning a number between 0 and 1 to the precision of the facts (the attribute *precision* of the fact *class*). Some examples of imprecision can be:
    * Rain: you can model the intensity of the rain: little rain, rains a lot ...
    * Probability of getting a head when tossing a coin
    * Cakes: I hate, dislike, like or love cakes...
* *Uncertainty of the rules*: in this case, some indetermination appears when the rule is created. Again, this indetermination is modelled as a number between 0 and 1 and in the **QRBS** is associated with the *certainty* of the rule (the attribute *certainty* of the rule class). Some examples of uncertainty can be:
    * When it rains I usually stay at home.
    * If the arterial pH is low there is a great probability that the patient suffers from Acidemia

So to add indetermination to our **QRBS** we need always set the **precision** attributes of the facts and the **certainty** attributes of the rules to a number between 0 and 1.

To deal with the indetermination in the **QRBS** NEASQC software library three different models were implemented:

1. **Certainty factors**. Proposed by Shortliffe and Buchanan in: *Shortliffe, E. H., & Buchanan, B. G. (1975). A model of inexact reasoning in medicine. Mathematical Biosciences,23(3), 351–379*. 
    * Available using the **BuilderImpl** class from **neasqc_qrbs.knowledge_rep** (or setting model to *cf* in the **execute** method of the **MyQlmQPU** object)
2. **Fuzzy logic**: introduced by Loft A. Zadeh in: *Zadeh, L. A. (1965). Fuzzy sets. Information and Control, 8(3), 338–353*
    * Available using the **BuilderFuzzy** class from **neasqc_qrbs.knowledge_rep** (or setting model to *fuzzy* in the **execute** method of the **MyQlmQPU** object)
3. **Bayesian networks**: following the work from:  *Borujeni, S. E., Nannapaneni, S., Nguyen, N. H., Behrman, E. C., & Steck, J. E. (2021). Quantum circuit representation of Bayesian networks. Expert Systems with Applications, 176, 114768*
    * Available using the **BuilderBayes** class from **neasqc_qrbs.knowledge_rep** (or setting model to *bayes* in the **execute** method of the **MyQlmQPU** object)
In this notebook we explain how to use **QRBS** to build easy and simple rule systems, implement the associated quantum circuits and execute in a Quantum Process Unit from Eviden myQLM.

In [ ]:
import sys
sys.path.append("../../")

## 1.Managing indetermination (example 1)

### Precision of the facts.

For dealing with the indetermination of the facts we are going to use the **precision** attribute of the facts objects.

As an example, we are going to implement the following indeterminate rules in the **QRBS** system:

*When It rains a lot I almost always stay at home. When I stay at home I usually read*

First, we are going to list the different facts of the system:

* Fact(Rain): in this case we are going to modulated the **intensity** of the rain. When precision is 0 then there is no rain, and when precision is 1 then it rains a lot. If rains moderately we can fix the probability to 0.5 for example
* Fact(Home): here we model the **probability** of stay at home. So for precision 0.0, we are not at home, and for precision 1.0 then we are at home surely. For example, a precision of 0.2 implies that there is a low probability of being at home.
* Fact(Read): again we model here the **probability** of being reading. Precision 0 I am not reading and 1.0 I am definitely reading. So if precision is 0.8 there is a high probability of being reading.


In [ ]:
from neasqc_qrbs.qrbs import QRBS

In [ ]:
qrbs =  QRBS()

In [ ]:
#facts

rain = qrbs.assert_fact("Rain", "Rain")
home = qrbs.assert_fact("Home", "Home")
read = qrbs.assert_fact("Read", "Read")

### Certainty of the rules

For dealing with the indetermination of the rules we are going to use the **certainty** attribute of the rule object.

The rules of our system are:

* Rule 1: *the rule is It rains a lot I almost always stay at home*. The rule has some uncertainty associated because is modulated by the *almost always* so we are going to set the certainty of the rule to 0.9 (because there is a little probability that It is raining a lot and I have to go out).
    * Left-hand side: Rain
    * Right-hand side: Home
    * certainty: 0.9
* Rule 2: *When I stay at home I usually read*. Again there is some certainty associated with the rule. In this case, we are going to fix it to 0.75.
    * Left-hand side: Home
    * Right-hand side: Read
    * certainty: 0.75

In [ ]:
#rules
rule1 = qrbs.assert_rule(rain, home)
rule1.certainty = 0.9
rule2 = qrbs.assert_rule(home, read)
rule2.certainty = 0.75

Now we need to chain the rules in a *knowledge island* because we want to know the probability of being reading!

In [ ]:
island = qrbs.assert_island([rule1, rule2])

We are going to suppouse that rains with a certain intensity: 0.8

In [ ]:
rain.precision = 0.8

Now we can use the different builders for getting a visualization of the quantum circuits in each of the indetermination propagation models:

In [ ]:
# The three Builders
from neasqc_qrbs.knowledge_rep import  BuilderImpl, BuilderBayes, BuilderFuzzy

In [ ]:
# Certainty factors engine
qc = island.build(BuilderImpl())
%qatdisplay qc --sv

In [ ]:
# Fuzzy Logic engine
qc = island.build(BuilderFuzzy())
%qatdisplay qc --sv

In [ ]:
# Bayesian Networks engine
qc = island.build(BuilderBayes())
%qatdisplay qc --sv

Now we can solve the system

In [ ]:
# myQLM qpus
from qpu.select_qpu import select_qpu
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config_c = {
    "qpu_type": ideal_qpus[0], 
}
qpu = select_qpu(qpu_config_c)


In [ ]:
from selectable_qpu import SelectableQPU
MyQlmQPU = SelectableQPU()

### model certainty facts

In [ ]:
# Certainty factors
rain.precision = 1.0
home.precision = 0.0
read.precision = 0.0
print("Precision of rain fact before execution is: ", rain.precision)
print("Precision of home fact before execution is: ", home.precision)
print("Precision of read fact before execution  is: ", read.precision)
MyQlmQPU.execute(qrbs, qpu=qpu, model='cf', shots=512)
print("Precision of rain fact after execution is: ", rain.precision)
print("Precision of home fact after execution is: ", home.precision)
print("Precision of read fact after execution  is: ", read.precision)

**BE AWARE** The number of shots will vary the output results!!

In [ ]:
# Certainty factors
rain.precision = 1.0
home.precision = 0.0
read.precision = 0.0
print("Precision of rain fact before execution is: ", rain.precision)
print("Precision of home fact before execution is: ", home.precision)
print("Precision of read fact before execution  is: ", read.precision)
MyQlmQPU.execute(qrbs, qpu=qpu, model='cf', shots=1024)
print("Precision of rain fact after execution is: ", rain.precision)
print("Precision of home fact after execution is: ", home.precision)
print("Precision of read fact after execution  is: ", read.precision)

#### model of fuzzy logic

In [ ]:
rain.precision = 1.0
home.precision = 0.0
read.precision = 0.0
print("Precision of rain fact before execution is: ", rain.precision)
print("Precision of home fact before execution is: ", home.precision)
print("Precision of read fact before execution  is: ", read.precision)
MyQlmQPU.execute(qrbs, qpu=qpu, model='fuzzy', shots=1024)
print("Precision of rain fact after execution is: ", rain.precision)
print("Precision of home fact after execution is: ", home.precision)
print("Precision of read fact after execution  is: ", read.precision)

#### bayesian networks

In [ ]:
# bayesian networks
rain.precision = 1.0
home.precision = 0.0
read.precision = 0.0
print("Precision of rain fact before execution is: ", rain.precision)
print("Precision of home fact before execution is: ", home.precision)
print("Precision of read fact before execution  is: ", read.precision)
MyQlmQPU.execute(qrbs, qpu=qpu, model='bayes', shots=1024)
print("Precision of rain fact after execution is: ", rain.precision)
print("Precision of home fact after execution is: ", home.precision)
print("Precision of read fact after execution  is: ", read.precision)

**We can get the probability of being reading if it rains enough precision=0.75**

In [ ]:
# Certainty factors
rain.precision = 0.75
home.precision = 0.0
read.precision = 0.0
print("Precision of rain fact before execution is: ", rain.precision)
print("Precision of home fact before execution is: ", home.precision)
print("Precision of read fact before execution  is: ", read.precision)
MyQlmQPU.execute(qrbs, qpu=qpu, model='cf', shots=512)
print("Precision of rain fact after execution is: ", rain.precision)
print("Precision of home fact after execution is: ", home.precision)
print("Precision of read fact after execution  is: ", read.precision)

**We can get the probability of being reading if it rains a little precision=0.2**

In [ ]:
# Certainty factors
rain.precision = 0.2
home.precision = 0.0
read.precision = 0.0
print("Precision of rain fact before execution is: ", rain.precision)
print("Precision of home fact before execution is: ", home.precision)
print("Precision of read fact before execution  is: ", read.precision)
MyQlmQPU.execute(qrbs, qpu=qpu, model='cf', shots=1024)
print("Precision of rain fact after execution is: ", rain.precision)
print("Precision of home fact after execution is: ", home.precision)
print("Precision of read fact after execution  is: ", read.precision)

## 2. Managing several islands (example 2)

A **RBS** can have several outputs instead of only one output. In this case the number of **knowledge islands** of our **QRBS** must be equal to the number of outputs of the system. So for each output an *island* must be created. 

As an example, we are going to implement the following situation:

*A family (father, mother, and daughter) is deciding what to do this weekend, and each one is suggesting a plan:*
1. Going to the grandparents' house: The father proposes this plan, which is a good option if it rains or if it's been a long time since they last visited.
2. Going to the cinema: The mother suggests this plan, which is also suitable for rainy days and if it has been a while since their last outing.
3. Going to the park: The daughter proposes this plan, which she knows can be done if it's not raining or if it's her birthday, and especially if they don't put something on TV that she likes, such as Paw Patrol or Bluey.

We are going to use the **QRBS** for modelling the situation:

**Facts**: We have the following input facts:
* Fact(Rain): binary fact: precision 0 for not raining.
* Fact(last_visited): model the time of the last visit to the grandfathers if precision is 0 then the last visit was recently and if is 1 then a lot of time has passed from the last visit.
* Fact(birthday): The birthday of the daughter only can be 0 (not birthday) or 1 (birthday)
* Fact(tv): if there is something that the daughter loves on tv (precision 1) or there is nothing (precision 0).

As output facts, we have three possible plans:
* Fact(grandfathers): visit grandfathers. 
* cinema: go to cinema. 
* park: go to park.

All of them will have a precision that models the suitability of the plan: 0 for not viability to 1 for very high viability. We are going to need 3 different *knowledge islands* for modelling them into the **QRBS** (one for each output fact).

In [ ]:
from neasqc_qrbs.knowledge_rep import AndOperator, OrOperator, NotOperator

In [ ]:
family =  QRBS()

In [ ]:
# inputs
rain = family.assert_fact("Rain", "Rain")
last_visit = family.assert_fact("last_visit", "last_visit")
birthday = family.assert_fact("birthday", "birthday")
tv = family.assert_fact("tv", "tv")

In [ ]:
#outputs
grandfathers = family.assert_fact("grandfathers", "grandfathers")
cinema = family.assert_fact("cinema", "cinema")
park = family.assert_fact("park", "park")

Now we need to create the rules of the **QRBS**:

* Father Rule: Rule((fact(rain) or fact(last_visit), grandfathers). The certainty will be set to 1.0
* Mather Rule: Rule(fact(rain) and not(fact(last_visit)), cinema). The certainty will be set to 1.0
* Daughter Rule: Rule(and( (not rain or birthday)) and not tv,  park).  The certainty will be set to 1.0

In [ ]:
rule1 = family.assert_rule(OrOperator(rain, last_visit), grandfathers)
rule1.certainty = 1.0
rule2 = family.assert_rule(AndOperator(rain, NotOperator(last_visit)), cinema)
rule2.certainty = 1.0

rule3 = family.assert_rule(
    AndOperator(
        OrOperator(NotOperator(rain), birthday),
        NotOperator(tv)
    ),
    park
)
rule3.certainty = 1.0


In this case we want a viability puntuation for each of the three posible plans: *grandfathers, cinema or parks* so for each desired output we need to create a *knowledge island*:

In [ ]:
father_plan = family.assert_island([rule1])
mother_plan = family.assert_island([rule2])
daughter_plan = family.assert_island([rule3])

In [ ]:
rain.precision = .8
last_visit.precision = 0.7
birthday.precision = 0.0 
tv.precision = 0.0

In [ ]:
MyQlmQPU.execute(family, qpu=qpu)

In [ ]:
print("grandfathers viability: ", grandfathers.precision)
print("cinema viability: ", cinema.precision)
print("park viability: ", park.precision)